In [1]:
import pickle
import numpy as np
import torch
from utils import *
from tqdm.notebook import trange, tqdm
import random
import json
import seaborn as sns
from sklearn.metrics import top_k_accuracy_score, confusion_matrix
import pandas as pd

In [2]:
from classes import CLASSES, GENERIC_PROMPT_COLLECTIONS
import torch.nn as nn
from clip import clip
import os
import torch


# Initialize CLIP models 
class TextEncoder(nn.Module):
    def __init__(self, clip_model):
        super().__init__()
        self.transformer = clip_model.transformer
        self.positional_embedding = clip_model.positional_embedding
        self.ln_final = clip_model.ln_final
        self.text_projection = clip_model.text_projection
        self.dtype = clip_model.dtype
        self.token_embedding = clip_model.token_embedding

    def forward(self, text):
        x = self.token_embedding(text).type(self.dtype)  # [batch_size, n_ctx, d_model]

        x = x + self.positional_embedding.type(self.dtype)
        x = x.permute(1, 0, 2)  # NLD -> LND
        x = self.transformer(x)
        x = x.permute(1, 0, 2)  # LND -> NLD
        x = self.ln_final(x).type(self.dtype)
        x = x[torch.arange(x.shape[0]), text.argmax(dim=-1)] @ self.text_projection

        return x

def load_clip_to_cpu(visual_backbone):
    backbone_name = visual_backbone
    url = clip._MODELS[backbone_name]
    model_path = clip._download(url, os.path.expanduser("~/.cache/clip"))

    try:
        # loading JIT archive
        model = torch.jit.load(model_path, map_location="cpu").eval()
        state_dict = None

    except RuntimeError:
        state_dict = torch.load(model_path, map_location="cpu")

    model = clip.build_model(state_dict or model.state_dict())

    return model

clip_model = load_clip_to_cpu("RN50")

visual_model = torch.nn.DataParallel(clip_model.visual).cuda()

text_model = TextEncoder(clip_model)
text_model = torch.nn.DataParallel(text_model).cuda()

Collect Text

In [3]:
desc_path = "/nethome/bdevnani3/flash1/long_tail_lang/data_loader/imagenet/wiki/desc_{}.txt"

all_labels_wiki_text = {}
all_labels_wiki_text_embs = {}

with torch.no_grad():
    for label in tqdm(range(1000)):
        all_labels_wiki_text[label] = []
        label_desc_path = desc_path.format(label)
        f = open(label_desc_path)
        for line in f:
            line = line.strip()
            if "==" in line:
                continue
            if len(line) == 0:
                continue
            all_labels_wiki_text[label].append(line[:76])
        texts = clip.tokenize(all_labels_wiki_text[label])
        texts = texts.cuda()
        zeroshot_weights = text_model(texts).float()
        zeroshot_weights = zeroshot_weights / zeroshot_weights.norm(
            dim=-1, keepdim=True
        )
        all_labels_wiki_text_embs[label] = zeroshot_weights
        f.close()

In [4]:
all_labels_text_embs = {}

with torch.no_grad():
    for label in tqdm(range(1000)):
        all_labels_text_embs[label] = []

        templates = np.array(GENERIC_PROMPT_COLLECTIONS["ImageNet"])
        c = np.array(CLASSES)[label]
            
        texts = clip.tokenize([template.format(c) for template in templates]) 
        texts = texts.cuda()
        zeroshot_weights = text_model(texts).float()
        zeroshot_weights = zeroshot_weights / zeroshot_weights.norm(
            dim=-1, keepdim=True
        )
        all_labels_text_embs[label].append(zeroshot_weights)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [5]:
cupl_texts = json.load(open("/nethome/bdevnani3/flash1/long_tail_lang/results_sklearn/cupl_image_prompts.json"))
cupl_embeds = {}
with torch.no_grad():
    for key in tqdm(cupl_texts):
        txts = clip.tokenize(cupl_texts[key])
        txts = txts.cuda()
        zeroshot_weights = text_model(txts).float()
        zeroshot_weights = zeroshot_weights / zeroshot_weights.norm(
            dim=-1, keepdim=True
        )
        idx = CLASSES.index(key)
        print(idx)
        cupl_embeds[idx] = zeroshot_weights

  0%|          | 0/998 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [8]:
print(all_labels_wiki_text_embs.keys())
print(all_labels_text_embs.keys())
print(cupl_embeds.keys())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [9]:
combined_text_embeds = {}
for key in range(1000):
    if key not in cupl_embeds:
        all_embs = torch.cat([all_labels_wiki_text_embs[key], all_labels_text_embs[key][0]], dim=0)
    else:
        all_embs = torch.cat([all_labels_wiki_text_embs[key], all_labels_text_embs[key][0], cupl_embeds[key]], dim=0)
        
    combined_text_embeds[key] = all_embs

In [10]:
import numpy as np
per_label_mu_sigma = {key: [None,None] for key in range(1000)}

for key in tqdm(range(1000)):
    data = combined_text_embeds[key]
    mu = np.mean(data.cpu().numpy(), axis=0)
    sigma = np.cov(data.cpu().numpy(), rowvar=0)
    per_label_mu_sigma[key] = [mu, sigma]
    
#     new_data_point = np.random.multivariate_normal(mean, covariance, 1)

In [12]:
# Get test data

from data_loader import dataloaders as dataloader
d = dataloader.load_data(
    data_root="../datasets/ImageNet/",
    dataset="ImageNet_LT",
    phase="test",
    batch_size=128,
#     batch_size=1,
    sampler_dic=None,
    num_workers=12,
    type="random_prompts",
    prompt_set="ImageNet",
)
data_test = d[0]

from classes import CLASSES, CUSTOM_TEMPLATES, GENERIC_PROMPT_COLLECTIONS
from tqdm.notebook import trange, tqdm

test_images, test_texts = [], []
test_labels = []

with torch.no_grad():
    for im, label, _, path in tqdm(data_test):
        x = visual_model(im.half()).float()
        x = x / x.norm(dim=-1, keepdim=True)
        test_images.append(x)
        test_labels.append(label)
        
test_images = torch.cat(test_images, dim=0)
test_labels = torch.cat(test_labels, dim=0)

lt_images = torch.load("clip_embedded_images.pt")
lt_images_labels = torch.load("clip_embedded_image_labels.pt")

freqs = {}
for label in lt_images_labels:
    label = int(label.item())
    if label not in freqs:
        freqs[label] = 0
    freqs[label] +=1
    
label_cats = {"many":[], "med":[], "few":[]}
for label in set(lt_images_labels):
    label = label.item()
    if freqs[label] > 100:
        label_cats["many"].append(label)
    elif freqs[label] > 20:
        label_cats["med"].append(label)
    else:
        label_cats["few"].append(label)
import json
# json.dump(freqs, open("/nethome/bdevnani3/flash1/long_tail_lang/embedding_datasets/class_frequencies.json", "w"))
# json.dump(label_cats, open("/nethome/bdevnani3/flash1/long_tail_lang/embedding_datasets/label_classification.json", "w"))

many_test_images, med_test_images, few_test_images = [], [], []
many_test_labels, med_test_labels, few_test_labels = [], [], []

for i,l in tqdm(zip(test_images, test_labels)):
    
    if l.item() in label_cats["many"]:
        many_test_images.append(i)
        many_test_labels.append(l.item())
    elif l.item() in label_cats["med"]:
        med_test_images.append(i)
        med_test_labels.append(l.item())
    elif l.item() in label_cats["few"]:
        few_test_images.append(i)
        few_test_labels.append(l.item())
        
many_test_images = torch.stack(many_test_images,dim=1)
med_test_images = torch.stack(med_test_images,dim=1)
few_test_images = torch.stack(few_test_images,dim=1)
many_test_labels = torch.tensor(many_test_labels)
med_test_labels = torch.tensor(med_test_labels)
few_test_labels = torch.tensor(few_test_labels)

print(many_test_images.shape, many_test_labels.shape)
print(med_test_images.shape, med_test_labels.shape)
print(few_test_images.shape, few_test_labels.shape)

Loading data from /nethome/bdevnani3/flash1/long_tail_lang/data/ImageNet_LT/ImageNet_LT_test.txt
Use data transformation: Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=None)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
)
***********************DATASET: test random_prompts
test 50000
No sampler.
Shuffle is True.


  0%|          | 0/391 [00:00<?, ?it/s]

0it [00:00, ?it/s]

torch.Size([1024, 19250]) torch.Size([19250])
torch.Size([1024, 23450]) torch.Size([23450])
torch.Size([1024, 7300]) torch.Size([7300])


In [13]:
data = []
labels = []

for k in combined_text_embeds:
    data.append(combined_text_embeds[k])
    for _ in range(len(combined_text_embeds[k])):
        labels.append(k)
        
data = torch.cat(data)
labels = torch.tensor(labels)
print(data.shape)
print(labels.shape)

torch.Size([175824, 1024])
torch.Size([175824])


In [25]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
json.dump(per_label_mu_sigma, open("per_label_mu_sigma.json", "w"), cls = NumpyEncoder)

In [21]:
from scipy.linalg import cholesky
from numpy.random import normal
import json


gauss_data = []
gauss_labels = []

per_label_mu_sigma = json.load("per_label_mu_sigma.json")

for k in tqdm(per_label_mu_sigma):
    mu, sigma = per_label_mu_sigma[k]
    for i in tqdm(range(200)):
#         l = cholesky(sigma, check_finite=False, overwrite_a=True)
#         new_data_point = mu + l.dot(standard_normal(len(means)))
        new_data_point = np.random.multivariate_normal(mu, sigma, 1)
#         new_data_point = normal(mu, sigma, len(mu))
        gauss_data.append(new_data_point)
        gauss_labels.append(k)
        
gauss_data = np.concatenate(gauss_data, axis=0)
gauss_labels = np.array(gauss_labels)


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [29]:
from dask.distributed import Client
from scipy.linalg import cholesky
from numpy.random import normal
import json

per_label_mu_sigma = json.load(open("per_label_mu_sigma.json", "r"))

client = Client(n_workers=10) # In this example I have 8 cores and processes (can also use threads if desired)

def my_function(i):
    mu, sigma = i
    out = []
    for _ in tqdm(range(200)):
        new_data_point = np.random.multivariate_normal(mu, sigma, 1)
        out.append([new_data_point, k])
    return out

futures = []

for i in per_label_mu_sigma:
    print(i)
    future = client.submit(my_function, per_label_mu_sigma[i])
    futures.append(future)

results = client.gather(futures)
np.savez("gaussian_texts", results)
client.close()

KeyboardInterrupt: 

In [ ]:
numpy.concatenate( LIST, axis=0 )
gauss_data = no.concatenate( LIST, axis=0 )
# gauss_labels = torch.tensor(gauss_labels)

In [16]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Seperability of the prompts 

clf = LogisticRegression(random_state=0, verbose=1, n_jobs=-1).fit(data.cpu(), labels)
print(clf.score(data.cpu(), labels))
print(clf.score(lt_images.cpu(), lt_images_labels.cpu()))
print(clf.score(test_images.cpu(), test_labels.cpu()))

print("Many:", clf.score(many_test_images.T.cpu(), many_test_labels.cpu()))
print("Med:", clf.score(med_test_images.T.cpu(), med_test_labels.cpu()))
print("Few:", clf.score(few_test_images.T.cpu(), few_test_labels.cpu()))
print("All:", clf.score(test_images.cpu(), test_labels.cpu()))